### Reads in data frame (from pickle!)
### Subsets data frame
### In future will get lots of lyrics from mxm

In [21]:
# IMPORTANT: edit this cell with your API info for billboard and spotify
mxm_key = "f8a009fe01e53de1e8a1388e60812ef5"

In [36]:
# Import everything we'll need in this notebook here
import billboard
from billboard import ChartData
import pdb 
import numpy as np 
import pandas as pd

# for lyrics stuff (mxm)
from musixmatch.track import TracksCollection
from musixmatch.lyrics import Lyrics

# for fuzzy matching artist names (when matching hot100 song to lyrics in mxm)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# for joining list of strings
import string


# for ascii / utf8 errors, run this
import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
print 'testing!'

import glob


testing!


In [48]:
#before we read in the dataframe, let's get a list of dates included in the billboard dataset
#note this will only work if you have all the csv files that steph posted to google drive

file_list = glob.glob("./hot100/hot100*csv") #or wherever your csvs are stored
print 'num lists = ', len(file_list)

#
all_dates = [] #here we will store all the dates
date_ind_start = 16
date_ind_end = 26
for f in file_list:
    all_dates.append(f[date_ind_start:date_ind_end])
print 'first 4 dates are : ', all_dates[0:4]

# make another list 'monthly_dates' that only stores every 4th date from all_dates
monthly_dates = all_dates[0::4]
print 'subsetting to monthly gives ', len(monthly_dates), ' lists'
print 'first 4 dates of monthly subset are : ', monthly_dates[0:4]


num lists =  3023
first 4 dates are :  ['1958-08-09', '1958-08-16', '1958-08-23', '1958-08-30']
subsetting to monthly gives  756  lists
first 4 dates of monthly subset are :  ['1958-08-09', '1958-09-06', '1958-10-04', '1958-11-01']


In [49]:
df = pd.read_pickle('./Billboard100DF')

In [50]:
df.head()

,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09


In [65]:
## let's subset to monthly data using our monthly date list.
print 'rows in full dataframe = ', len(df)

# save monthly dataframe
dfmonthly = df[df['date'].isin(monthly_dates)].reindex()


print 'rows in monthly subset = ', len(dfmonthly)
print 'ratio = ', len(dfmonthly)/float(len(df))
dfmonthly.head()



rows in full dataframe =  302142
rows in monthly subset =  75563
ratio =  0.250091016807


,Artist,Change,LastPos,PeakPos,Rank,SpotifyID,SpotifyLink,Title,Weeks,date
0,Ricky Nelson,Re-Entry,0,1,1,33FPsMEl3UwpytDuyf9VYq,https://embed.spotify.com/?uri=spotify:track:3...,Poor Little Fool,11,1958-08-09
1,Perez Prado And His Orchestra,Re-Entry,0,2,2,0nG4SbrAONIFLIwQqegooj,https://embed.spotify.com/?uri=spotify:track:0...,Patricia,14,1958-08-09
2,Bobby Darin,Re-Entry,0,3,3,6NCPbpsr2N4w6YIZkpq6jx,https://embed.spotify.com/?uri=spotify:track:6...,Splish Splash,8,1958-08-09
3,Elvis Presley With The Jordanaires,Re-Entry,0,4,4,0MLachoG7wHVRrVydw2xuJ,https://embed.spotify.com/?uri=spotify:track:0...,Hard Headed Woman,8,1958-08-09
4,Kalin Twins,Re-Entry,0,5,5,52l2PCOwU2n2zfTlc0pEov,https://embed.spotify.com/?uri=spotify:track:5...,When,9,1958-08-09


### Note the rows in this new subsetted dataframe still have the indices from the old dataframe. Hopefully that's ok

In [23]:
# make function that takes a list of artists (from mxm), and selects the best match to the artist from hot100
def find_artist_match(name_to_match, artist_list):
    index = 0 #index of the artist we're scoring
    r = 0 #here i'll store the highest similarity score
    for a in artist_list:
        rnew = fuzz.ratio(name_to_match, a)
        print 'similarity between ', name_to_match, ' and ', a, ' : ', rnew
        if rnew > r: 
            best_match = a
            best_ind = index
            r = rnew
        index = index+1

    print 'Best Match = ', best_match
    assert(best_match==artist_list[best_ind])
    print 'Score = ',r
    return best_ind


In [66]:
df['Title'][0]

'Poor Little Fool'

In [26]:
apikey = mxm_key
num_songs = 3 #how many songs to get from the hot100 song list

df_mxm_info = pd.DataFrame() #dataframe for some song info (from collections object)
#df_mxm_lyrics = pd.DataFrame() #dataframe for lyrics and related attributes
lyrics_list = [] #append lyric info to a list for each song, later we'll put it in a dataframe!


#header for info to be stored in df_mxm_info
cols = ['lyrics_id', 'has_lyrics', 'artist_name', 'track_spotify_id','track_id','track_name']

#header for info in df_mxm_lyrics
lyrics_df_cols = ['lyrics_copyright', 'lyrics_body','publisher_list', 'can_edit', 'writer_list', \
                  'instrumental','verified','html_tracking_url','lyrics_language_description', \
                  'locked','restricted','explicit','updated_time','lyrics_id','script_tracking_url', \
                  'action_requested','pixel_tracking_url','lyrics_language']

for i in xrange(num_songs):
    trackname_hot = df['Title'][i]
    print 'trackname hot = ',trackname_hot
    artist_hot = df['Artist'][i]
    print 'artist hot = ', artist_hot

    collection = TracksCollection.fromSearch(q_track=trackname_hot, apikey = apikey)

    # print artists that have that title:
    artists = [t['artist_name'] for t in collection]
    print 'There are ', len(artists), ' artists with this song title'
    print artists
    track_id = [t['track_id'] for t in collection]

    # fuzzy match the artist from hot100 to all artists matching track name
    ind = find_artist_match(artist_hot, artists)
    print artists[ind]

    # select the best track id (best match to artist name)
    track_id_selected = track_id[ind]
    print track_id_selected
    
    # store only one row in df_mxm_info
    # this row represents the best match to the billboard title+artist
    subset = pd.DataFrame.from_dict(collection)[cols] #could set index here to track_id or lyrics_id...
    is_true = subset['track_id'] == track_id_selected
    df_mxm_info = df_mxm_info.append(subset[is_true])
    
    # now add to the lyrics data frame as well:
    lyrics = Lyrics(track_id = track_id_selected, apikey = apikey)
    
    #df_mxm_lyrics = df_mxm_lyrics.append(lyrics.values())
    lyrics_list.append(lyrics.values())

trackname hot =  Poor Little Fool
artist hot =  Ricky Nelson
There are  10  artists with this song title
[u'Ricky Nelson', u'Brian Hyland', u'The Fleetwoods', u'Rick Nelson', u'Bill Dennis', u'The Slapbacks', u'Helen Reddy', u'Ike & Tina Turner', u'Raul Seixas', u'Raul Seixas']
similarity between  Ricky Nelson  and  Ricky Nelson  :  100
similarity between  Ricky Nelson  and  Brian Hyland  :  33
similarity between  Ricky Nelson  and  The Fleetwoods  :  23
similarity between  Ricky Nelson  and  Rick Nelson  :  96
similarity between  Ricky Nelson  and  Bill Dennis  :  35
similarity between  Ricky Nelson  and  The Slapbacks  :  24
similarity between  Ricky Nelson  and  Helen Reddy  :  26
similarity between  Ricky Nelson  and  Ike & Tina Turner  :  21
similarity between  Ricky Nelson  and  Raul Seixas  :  35
similarity between  Ricky Nelson  and  Raul Seixas  :  35
Best Match =  Ricky Nelson
Score =  100
Ricky Nelson
8921584
trackname hot =  Patricia
artist hot =  Perez Prado And His Orches

In [28]:
df_mxm_info

,lyrics_id,has_lyrics,artist_name,track_spotify_id,track_id,track_name
0,7949983,1,Ricky Nelson,6kGo2CzDxnPP6pxbqABZ7l,8921584,Poor Little Fool
3,0,0,Perez Prado And His Orchestra,1rkjy666GnUZDTzyBd7fpq,8978935,Patricia
1,1274168,1,Bobby Darin,4RxtXFzUxQCOBAbggjN7mM,18803546,Splish Splash


In [29]:
#check that lyrics list is same length as info df
len(lyrics_list) == len(df_mxm_info)

True

In [30]:
# Store the lyrics list elements in a data frame
df_mxm_lyrics = pd.DataFrame(data= lyrics_list, columns=lyrics_df_cols) 
df_mxm_lyrics

,lyrics_copyright,lyrics_body,publisher_list,can_edit,writer_list,instrumental,verified,html_tracking_url,lyrics_language_description,locked,restricted,explicit,updated_time,lyrics_id,script_tracking_url,action_requested,pixel_tracking_url,lyrics_language
0,Lyrics powered by www.musiXmatch.com. This Lyr...,I used to play around with hearts\nThat hasten...,[],0,[],0,0,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,0,0,0,2013-02-01T20:05:11Z,7949983,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
1,,,[],0,[],1,0,,,0,0,0,2015-04-02T23:55:38Z,11100045,,,,
2,Lyrics powered by www.musiXmatch.com. This Lyr...,"Splish splash, I was taking a bath\nLong about...",[],0,[],0,0,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,0,0,0,2010-07-06T21:09:14Z,1274168,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en


In [33]:
# merge these 2 dataframes
# note that the song with no lyrics is exclued from the final data frame. this makes sense?
df_mxm = pd.merge(df_mxm_info,df_mxm_lyrics,on='lyrics_id')
df_mxm

,lyrics_id,has_lyrics,artist_name,track_spotify_id,track_id,track_name,lyrics_copyright,lyrics_body,publisher_list,can_edit,...,html_tracking_url,lyrics_language_description,locked,restricted,explicit,updated_time,script_tracking_url,action_requested,pixel_tracking_url,lyrics_language
0,7949983,1,Ricky Nelson,6kGo2CzDxnPP6pxbqABZ7l,8921584,Poor Little Fool,Lyrics powered by www.musiXmatch.com. This Lyr...,I used to play around with hearts\nThat hasten...,[],0,...,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,0,0,0,2013-02-01T20:05:11Z,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
1,1274168,1,Bobby Darin,4RxtXFzUxQCOBAbggjN7mM,18803546,Splish Splash,Lyrics powered by www.musiXmatch.com. This Lyr...,"Splish splash, I was taking a bath\nLong about...",[],0,...,http://tracking.musixmatch.com/t1.0/m_html/e_1...,English,0,0,0,2010-07-06T21:09:14Z,http://tracking.musixmatch.com/t1.0/m_js/e_1/s...,,http://tracking.musixmatch.com/t1.0/m_img/e_1/...,en
